# Module

In [2]:
# From the repository
from util import *
from read_data import *
data_names = list(name2file_name.keys())
print(data_names)

# Basic modules
import os
import glob
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from infomap import Infomap


#name = "wiod2016"
name = data_names[3]
print("We are going to use: " + name)
# Enable original_format to use the format as originally provided
data_dict = get_data(name,original_format=False)
data_dict.keys()
#df_nodes = data_dict["df_nodes"]
df_edges = data_dict["df_edges"]

['blogcatalog', 'homosapiens', 'wikipos', 'enron', 'unvote', 'untrade', 'uslegis_net', 'uslegis_net_small_dyn', 'uslegis_net_dyn', 'uslegis_hyp_dyn', 'contacts', 'dawn_net', 'dawn_hyp', 'ndc_net', 'ndc_hyp', 'coauth_dblp_net', 'coauth_dblp_hyp', 'wiod2016', 'wiod2013', 'wiodlong', 'eth', 'bitcoinalpha', 'bitcoinotc', 'uscourt']
We are going to use: enron


# Get rid of duplicates

In [3]:
df_edges = df_edges[["source","target"]].copy()
df_edges.drop_duplicates(inplace=True)

# Infomap

In [4]:
im = Infomap("--two-level --directed")

In [5]:
for i in range(len(df_edges)):
    if df_edges["source"].iloc[i] != df_edges["target"].iloc[i]:
        im.add_link(df_edges["source"].iloc[i],df_edges["target"].iloc[i])

In [6]:
%%time
im.run()

  Infomap v2.7.1 starts at 2023-10-19 23:38:19
  -> Input network: 
  -> No file output!
  -> Configuration: two-level
                    directed
  OpenMP 201511 detected with 72 threads...
  -> Ordinary network input, using the Map Equation for first order network flows
Calculating global network flow using flow model 'directed'... CPU times: user 2.51 s, sys: 0 ns, total: 2.51 s
Wall time: 39.4 ms

  -> Using unrecorded teleportation to links. 
  -> PageRank calculation done in 200 iterations.

  => Sum node flow: 1, sum link flow: 1
Build internal network with 182 nodes and 3007 links...
  -> One-level codelength: 7.23652232

Trial 1/1 starting at 2023-10-19 23:38:19
Two-level compression: 4.3% 1.4% 

In [7]:
print(f"Found {im.num_top_modules} modules with codelength: {im.codelength}")

print("Result")
print("\n#node module")
out = []
for node in im.tree:
    if node.is_leaf:
        out.append([node.node_id,node.module_id])

Found 9 modules with codelength: 6.823336260489542
Result

#node module


# Write results

In [9]:
df_info = pd.DataFrame(out)
df_info.columns = ["Id","info_group"]

In [10]:
df_info.to_csv("./tables/df_info.csv",index=False)